# Training an image classification model for predicting AD diagnosis

### Install necessary libraries

In [ ]:
!pip install --upgrade --quiet ultralytics==8.0.112 clearml tensorboard

In [ ]:
#!pip install opencv-python

!pip install opencv-python==4.8.0.76


  Obtaining dependency information for opencv-python==4.8.0.76 from https://files.pythonhosted.org/packages/f5/d0/2e455d894ec0d6527e662ad55e70c04f421ad83a6fd0a54c3dd73c411282/opencv_python-4.8.0.76-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 29.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.6.0.66
    Uninstalling opencv-python-4.6.0.66:
      Successfully uninstalled opencv-python-4.6.0.66


### Use ClearML for tracking

In [ ]:
"""import clearml
clearml.browser_login()"""

'import clearml\nclearml.browser_login()'

In [ ]:
"""#connect to google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

#!pip install --upgrade --quiet ultralytics==8.0.112 clearml tensorboard

#path = '/content/gdrive/MyDrive/car_detection_training/Car_Detection_Data'
yaml_path = '/content/gdrive/MyDrive/Car_Inspection_Model_Training'"""

"#connect to google drive\nfrom google.colab import drive\ndrive.mount('/content/gdrive', force_remount=True)\n\n#!pip install --upgrade --quiet ultralytics==8.0.112 clearml tensorboard\n\n#path = '/content/gdrive/MyDrive/car_detection_training/Car_Detection_Data'\nyaml_path = '/content/gdrive/MyDrive/Car_Inspection_Model_Training'"

In [ ]:
import boto3
import botocore
import os

s3 = boto3.client("s3")
#s3 = boto3.resource("s3")

In [ ]:
#unzip file
import zipfile


with zipfile.ZipFile("/tmp/data.zip", "r") as zip_ref:
    #zip_ref.extractall("/tmp/unzipped")
    #zip_ref.extractall(f"./{dest_folder_name}/tmp/unzipped")
    zip_ref.extractall("/home/ec2-user/SageMaker/datasets/Alzheimers-Image-Classification/tmp/unzipped")

In [ ]:
data_path = '/home/ec2-user/SageMaker/Alzheimers-Image-Classification/tmp/unzipped/data'

In [ ]:
#!pip install torch
#!conda install pytorch

In [ ]:
import cv2
import ultralytics
from ultralytics import YOLO

from datetime import datetime
curr_date = datetime.now().date()
print(curr_date)

Matplotlib is building the font cache; this may take a moment.


2023-10-19


In [ ]:
#Initialize clearml task
from clearml import Task, StorageManager, Dataset


task = Task.init(project_name = "mri_classification_model", task_name = f"mri_classification_model_{curr_date}")

In [ ]:
#get yolo classification model
yolo = YOLO("yolov8n-cls.pt")

100%|██████████| 5.28M/5.28M [00:00<00:00, 58.4MB/s]


In [ ]:
#train the model
logger = task.get_logger()

!yolo task=classify mode=train model=yolov8n-cls.pt data='./Alzheimers-Image-Classification/tmp/unzipped/data' epochs=100 imgsz = 640


New https://pypi.org/project/ultralytics/8.0.200 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.112 🚀 Python-3.10.12 torch-2.0.1 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=./Alzheimers-Image-Classification/tmp/unzipped/data, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnost

"\nlogger = task.get_logger()\n\n# in your training loop\nfor epoch in range(epochs):\n    # train your model\n    # ...\n    \n\n    # log your metrics\n    logger.report_scalar('loss', loss, step=epoch)\n    logger.report_scalar('accuracy', accuracy, step=epoch)\n"

In [ ]:
#Close ClearML task
task.close()

In [ ]:
#perform validation

trained_model = YOLO("runs/classify/train12/weights/best.pt")

metrics = trained_model.val()
print(metrics.top1, metrics.top5)

Ultralytics YOLOv8.0.112 🚀 Python-3.10.12 torch-2.0.1 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-cls summary (fused): 73 layers, 1440004 parameters, 0 gradients
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
               classes   top1_acc   top5_acc:   1%|          | 1/82 [00:00<00:52,  1.54it/s]Exception in thread Thread-40 (plot_images):
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/ultralytics/yolo/utils/plotting.py", line 384, in plot_images
    annotator.text((x, y), f'{c}', txt_color=color, box_style=True)
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/ultralytics/yolo/utils/plotting.py", line 2

0.6812933087348938 0.9999999403953552


In [ ]:
print(metrics.top1, metrics.top5)

0.6812933087348938 0.9999999403953552


In [ ]:
#predict single image
from PIL import Image

detections = trained_model(Image.open("./datasets/26 (19).jpg"))


0: 640x640 MildDemented 0.51, VeryMildDemented 0.48, ModerateDemented 0.00, NonDemented 0.00, 3.1ms
Speed: 9.0ms preprocess, 3.1ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
"""for detection in detections:

    probabs = detection.probs
    predictions = probabs.classes
    confidences = probabs.scores

    for detection in detections:
        pass

    print(predictions, confidences)"""

'for detection in detections:\n\n    probabs = detection.probs\n    predictions = probabs.classes\n    confidences = probabs.scores\n\n    for detection in detections:\n        pass\n\n    print(predictions, confidences)'